In [10]:
import spacy
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import matplotlib.pyplot as plt
from keras.utils import plot_model,multi_gpu_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import Model
from keras.models import load_model
from keras.layers import Input,GlobalAveragePooling2D,Layer,InputSpec
from keras.layers.core import Dense,Flatten,Dropout
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.utils import np_utils
import keras.backend as K
import keras.layers as kl
from keras.preprocessing import sequence
import tensorflow as tf
from tensorflow.python.client import device_lib
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from time import time
import pickle
from keras.models import model_from_json

# from AttentionModule import SelfAttention, SoftAttention
import os
visible_gpu_devices = '0'
os.environ["CUDA_VISIBLE_DEVICES"]=visible_gpu_devices
import cv2
from tqdm.notebook import tqdm
import AttentionMed as AM
from importlib import reload
reload(AM)
from time import time,localtime,strftime
from ipywidgets import IntProgress
# from coord import CoordinateChannel2D

In [11]:
n_gpu=len(visible_gpu_devices.split(','))
n_cpu=1
tf_config= tf.ConfigProto(device_count = {'GPU': n_gpu , 'CPU': n_cpu})
tf_config.gpu_options.allow_growth=True
s=tf.Session(config=tf_config)
K.set_session(s)

In [12]:
timesteps = 40
vocab_size = 100
embedding_size = 300
elu_alpha = 1.0
bottleneck_units = 512
batchsize = 256
def temp_datagen(batchsize=1):
    counter=0
    while True:
        inp = np.random.randint(1,vocab_size,(batchsize,timesteps,))
        out = inp.copy()
        out[:,::2]=0
        out = np_utils.to_categorical(out,num_classes=101)
        counter+=1
        if batchsize==counter:
            yield inp,out
            counter=0
t_datagen = temp_datagen(batchsize)

In [13]:
t_x,t_y=next(t_datagen)
t_x[:2],t_y.argmax(-1),t_y.shape

(array([[45, 84, 69, 18, 69, 94, 10, 88, 39,  9, 36, 70, 28, 83, 51, 42,
         60, 32, 90,  7, 86, 10,  9, 57, 84, 21, 81, 85, 74,  4,  7, 69,
         99, 95, 31, 85, 57, 17, 99, 80],
        [83,  7, 65, 29, 93, 67, 67, 17, 93, 96,  6,  1, 68, 17, 57, 61,
         13, 89, 72, 18, 89, 68, 19,  8, 57, 72, 34, 60, 53, 29, 99, 42,
         50, 81, 70, 45, 54, 45, 34, 73]]),
 array([[ 0, 84,  0, ..., 17,  0, 80],
        [ 0,  7,  0, ..., 45,  0, 73],
        [ 0, 88,  0, ..., 16,  0, 78],
        ...,
        [ 0, 46,  0, ..., 36,  0, 23],
        [ 0, 55,  0, ..., 60,  0, 83],
        [ 0, 37,  0, ..., 62,  0,  2]]),
 (256, 40, 101))

In [14]:
units = 512

inp = kl.Input(shape=(timesteps,))
emb = kl.Embedding(vocab_size+1, embedding_size, mask_zero=False, name='w2v_emb')(inp)
encoder = kl.LSTM(units,activation='relu',return_sequences=True)(emb)

# encoder,sa_maps = AM.SelfAttention(ch=512)(encoder)

# context_vector = kl.LSTM(512,activation='relu',return_sequences=False)(encoder)
context_vector = kl.GlobalAveragePooling1D(name='gap')(encoder)

acv_1,a_map_1 = AM.Attention(ch=512,timesteps=1,name='a1')([context_vector,encoder])
acv_1 = kl.BatchNormalization()(acv_1)
acv_2,a_map_2 = AM.Attention(ch=512,timesteps=1,name='a2')([context_vector,encoder])
acv_2 = kl.BatchNormalization()(acv_2)
acv_3,a_map_3 = AM.Attention(ch=512,timesteps=1,name='a3')([context_vector,encoder])
acv_3 = kl.BatchNormalization()(acv_3)
acv_4,a_map_4 = AM.Attention(ch=512,timesteps=1,name='a4')([context_vector,encoder])
acv_4 = kl.BatchNormalization()(acv_4)
acv_5,a_map_5 = AM.Attention(ch=512,timesteps=1,name='a5')([context_vector,encoder])
acv_5 = kl.BatchNormalization()(acv_5)
acv_6,a_map_6 = AM.Attention(ch=512,timesteps=1,name='a6')([context_vector,encoder])
acv_6 = kl.BatchNormalization()(acv_6)
acv_7,a_map_7 = AM.Attention(ch=512,timesteps=1,name='a7')([context_vector,encoder])
acv_7 = kl.BatchNormalization()(acv_7)
acv_8,a_map_8 = AM.Attention(ch=512,timesteps=1,name='a8')([context_vector,encoder])
acv_8 = kl.BatchNormalization()(acv_8)

concat_attention = kl.Concatenate(name='concat_attn')([acv_1,acv_2,acv_3,acv_4,acv_5,acv_6,acv_7,acv_8])
concat_attention = AM.CondenseAttention1D(ch_in=int(concat_attention.shape[-1]),ch_out=512)(concat_attention)
concat_attention = kl.Lambda(lambda x:K.squeeze(x,axis=1),name="attention_squeeze")(concat_attention)
concat_attention, g1, g2 = AM.ResidualCombine(method='add',name='residual_combine')([context_vector, concat_attention])

# acv_combine = kl.ELU(alpha=elu_alpha)(concat_attention)
# acv_combine = kl.Lambda(lambda x:K.expand_dims(x,axis=1),name="Expand")(acv_combine)
# acv_combine = kl.Conv1D(filters=bottleneck_units,kernel_size=1,strides=1,padding='same')(acv_combine)
# acv_combine = kl.BatchNormalization()(acv_combine)
# acv_combine = kl.Lambda(lambda x:K.squeeze(x,axis=1),name="Squeeze2")(acv_combine)

# # # # print(attended_context_vector)
repeat_context_vector = kl.RepeatVector(n=timesteps)(concat_attention)
decoder = kl.LSTM(512,activation='relu',return_sequences=True)(repeat_context_vector)
# # AM.Attention(ch=1)([])
decoder_out = kl.TimeDistributed(Dense(vocab_size+1,activation='softmax'))(decoder)
model = Model(inp,decoder_out)
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam(lr=0.001))

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
w2v_emb (Embedding)             (None, 40, 300)      30300       input_5[0][0]                    
__________________________________________________________________________________________________
lstm_8 (LSTM)                   (None, 40, 512)      1665024     w2v_emb[0][0]                    
__________________________________________________________________________________________________
gap (GlobalAveragePooling1D)    (None, 512)          0           lstm_8[0][0]                     
__________________________________________________________________________________________________
a1 (Attent

In [ ]:
# /media/MyDataStor1/mshaikh2/project_xray/notebooks
hist = model.fit_generator(t_datagen,epochs=1000,verbose=1,steps_per_epoch=50)

Epoch 1/1000
50/50 [==============================] - 27s 535ms/step - loss: 3.6136 - acc: 0.4901
Epoch 2/1000
50/50 [==============================] - 10s 192ms/step - loss: 2.6884 - acc: 0.5021
Epoch 3/1000
50/50 [==============================] - 10s 202ms/step - loss: 2.3001 - acc: 0.5066
Epoch 4/1000
50/50 [==============================] - 10s 204ms/step - loss: 2.2809 - acc: 0.5101
Epoch 5/1000
50/50 [==============================] - 11s 225ms/step - loss: 2.2661 - acc: 0.5118
Epoch 6/1000
50/50 [==============================] - 20s 409ms/step - loss: 2.2550 - acc: 0.5135
Epoch 7/1000
50/50 [==============================] - 21s 410ms/step - loss: 2.2456 - acc: 0.5145
Epoch 8/1000
50/50 [==============================] - 20s 409ms/step - loss: 2.2366 - acc: 0.5150
Epoch 9/1000
50/50 [==============================] - 20s 410ms/step - loss: 2.2300 - acc: 0.5158
Epoch 10/1000
50/50 [==============================] - 20s 409ms/step - loss: 2.2270 - acc: 0.5164
Epoch 11/1000
50/50

In [ ]:
df = pd.DataFrame(hist.history)
df.to_csv("graphs/gap_8attn_bn_res_add_metrics.csv")
plot1,plot2 = df.plot(subplots=True, figsize=(5,10))
fig1 = plot1.get_figure()
fig2 = plot2.get_figure()
fig1.savefig("graphs/gap_8attn_bn_res_add_acc.png")
fig2.savefig("graphs/gap_8attn_bn_res_add_loss.png")